In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("df_api").getOrCreate()

In [2]:
training=spark.read.csv('test2.csv',header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [3]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
training.columns

['Name', 'age', 'Experience', 'Salary']

[Age,Experience]---->new feature---->independednt feature

In [5]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['age','Experience'],outputCol='Independent Feature')


In [6]:
output=featureassembler.transform(training)

In [7]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [8]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Feature']

In [9]:
final_df=output.select("Independent Feature","Salary")

In [10]:
final_df.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [11]:
from pyspark.ml.regression import LinearRegression

train_data,test_data=final_df.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Feature",labelCol='Salary')
regressor=regressor.fit(train_data)

In [12]:
regressor.coefficients

DenseVector([1771.9298, -298.2456])

In [13]:
regressor.intercept

-21912.28070175451

In [15]:
pred_results=regressor.evaluate(test_data)

In [16]:
pred_results.predictions.show()

+-------------------+------+------------------+
|Independent Feature|Salary|        prediction|
+-------------------+------+------------------+
|         [29.0,4.0]| 20000|28280.701754385987|
|         [30.0,8.0]| 25000|28859.649122807026|
+-------------------+------+------------------+



In [17]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(6070.175438596507, 41733456.44813812)